In [6]:
from pysr import PySRRegressor
#import simpy as sp
import os
import yaml
import numpy as np

import json


#PATH = '/home/jpierre/v2/pySr/data/mod1/data.json'
#PATH_CFG = '/master/code/experimental/pysr/cfg_pysr'
PATH = '/master/code/experimental/pysr/data/data/mod_1-lim0.json'


NB_RUN = 1000
BINARY_OP = ["+", "*", 'cond']
UNARY_OP = ["inv(x) = 1/x"]
VARIABLES = ['v_x_0', 'v_y_0', 'v_x_1', 'v_y_1', 'v_x_2', 'v_y_2', 'v_x_3', 'v_y_3', 'd', 'delta_x', 'delta_y', 'r_i', 'r_j']
MAX_SIZE = 11
#PARSIMONY = 0.0000032



def readJson(filePath:str):
    """
    Function to read json 
    """
    
    with open(filePath, 'r') as f:
        data = json.load(f)
    return data


def writeJson(data, filePath):
    """
    Function to write json 
    """
    with open(filePath, 'w') as f:
        json.dump(data, f, indent=2)


def getData(jsonPath = PATH):
    # for now


    messages = np.array(readJson(jsonPath)['messages'])
    edges = np.array(readJson(jsonPath)['edges'])[:, :3]

    return edges, messages



def getPySrModel(nbRun = NB_RUN, binaryOp = BINARY_OP, unaryOp = UNARY_OP, maxsize = MAX_SIZE):

    model = PySRRegressor(
        niterations=nbRun,
        binary_operators=binaryOp,
        #unary_operators= ["inv(x) = 1/x",],
        #populations=15,
        model_selection = "best",
        maxsize = maxsize,
        #complexity_of_variables = 2,
        parsimony = PARSIMONY,
        #nested_constraints = {"cond": {"*":0}},
        #adaptive_parsimony_scaling = 1000,
        #ncycles_per_iteration = 1000,
        #turbo = True,
        #extra_sympy_mappings = {"inv": lambda x: 1 / x}
        elementwise_loss = 'L1DistLoss()'
    )

    return model


def fittingModel(model, X, y, verbose:bool = False, variables = VARIABLES):

    if verbose:
        print(">>>>> Fitting pySr")


    # Fit model
    model.fit(X, y, 
    variable_names = variables,
    # X_units = [""],
    # y_units = "",
    )


    return model

In [7]:
a = readJson(PATH)

In [8]:
print(list(a.keys()))

['edges', 'messages']


In [9]:
X, Y = getData()

In [10]:
pyReg = getPySrModel()

mod = fittingModel(pyReg, X, Y, variables=[' r ', ' cosine ', ' sine '])

/usr/local/lib/python3.11/site-packages/pysr/sr.py:1307: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/pysr/sr.py:1387: UserWarning: Spaces in `variable_names` are not supported. Spaces have been replaced with underscores. 
Please use valid names instead.
  warnings.warn(


Compiling Julia backend...


[ Info: Started!



Expressions evaluated per second: 3.740e+04
Head worker occupation: 9.2%
Progress: 83 / 30000 total iterations (0.277%)
Best equations for output 1
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
3           8.717e-03  5.314e+00  y₀ = _cosine_ * -0.027244
5           8.717e-03  1.061e-05  y₀ = _cosine_ * (-0.6891 * 0.039352)
7           7.402e-03  8.177e-02  y₀ = -0.029117 * ((0.23887 * _sine_) + _cosine_)
9           7.401e-03  3.910e-05  y₀ = -0.029124 * (_cosine_ + (0.23842 * (-0.029124 + _sine_)))
11          7.401e-03  1.180e-05  y₀ = -0.029124 * ((0.23842 * ((-0.029124 * 0.53722) + _sine_))...
                                   + _cosine_)
---------------------------------------------------------------------------------------------------
Best equations for output 2
Hall of Fame:
-----------------------------------------------------------------------------------------------

In [11]:
print(mod.latex_table(indices=None, precision=3, columns=['equation', 'complexity', 'loss', 'score']))

\usepackage{breqn}
\usepackage{booktabs}

...

\begin{table}[h]
\begin{center}
\begin{tabular}{@{}cccc@{}}
\toprule
Equation & Complexity & Loss & Score \\
\midrule
$y_{0} = 0.000110$ & $1$ & $0.0180$ & $0.0$ \\
$y_{0} = - 0.0271 _cosine_{}$ & $3$ & $0.00872$ & $0.363$ \\
$y_{0} = 8.55 \cdot 10^{-5} - 0.0272 _cosine_{}$ & $5$ & $0.00872$ & $1.20 \cdot 10^{-5}$ \\
$y_{0} = _cosine_{} \left(0.0744 _r_{} - 0.153\right)$ & $7$ & $0.00648$ & $0.148$ \\
\begin{minipage}{0.8\linewidth} \vspace{-1em} \begin{dmath*} y_{0} = _cosine_{} _r_{} \left(0.0552 _r_{} - 0.110\right) \end{dmath*} \end{minipage} & $9$ & $0.00643$ & $0.00381$ \\
\begin{minipage}{0.8\linewidth} \vspace{-1em} \begin{dmath*} y_{0} = \left(_cosine_{} + 0.293 _sine_{}\right) \left(0.0784 _r_{} - 0.158\right) \end{dmath*} \end{minipage} & $11$ & $0.000906$ & $0.980$ \\
\bottomrule
\end{tabular}
\end{center}
\end{table}

\begin{table}[h]
\begin{center}
\begin{tabular}{@{}cccc@{}}
\toprule
Equation & Complexity & Loss & Score \\
\